Parameters used in the code:

N: Number of sites in the transverse field Ising model.

This code can optimize 12 or higher sites because we are using pytorch with GPU support

B: B-field value for the target Hamiltonian, which can be zero or non-zero.

J_value: J coupling strength.

J_matrix_alpha: Power-law decay constant.

x: the best field used in modulated time evolution, the first half is $\lambda (t)$ while the second half is $B(t)$

number_of_layers: The number of steps in the modulated time evolution.


There are two final output files,
- f'QAOA_training_log_N={N}_B={B}_layers_{np.sum(p_list)}.txt'
- f'QAOA_checkpoint_log_N={N}_B={B}_layers_{np.sum(p_list)}.pt'

In [1]:
import numpy as np
# initialize variables
N = 12
B = 0.1
J_value = 1
J_matrix_alpha =  1
number_of_layers=10
x =np.array([0.4072185919308115, 1.2954691806813516, 0.5805573609066998, 0.8454189546075643, 1.2243197218332320, 2.1572222947038711, 2.5905765854250022, 2.7409062412055198, 2.0081536293189819, 0.9755810490605400, 4.9955913368774452, 1.7717120168593767, 1.0871728400196139, 0.7185240323450447, 0.5732536483420920, 0.4951459317676031, 0.4388398363271400, 0.3901464618527199, 0.3117549554184353, 0.1945036761838408])


In [2]:
import os
import torch
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from scipy.sparse.linalg import eigs
from scipy.sparse.linalg import eigsh
from scipy.interpolate import Akima1DInterpolator
from scipy.linalg import expm
import time
from scipy import sparse
import matplotlib.pyplot as plt
import pandas
import scipy
np.set_printoptions(precision=6)
# initialize variables
# Q is the dimension
Q = pow(2, N)

# decimal number to binary array function
def D2B(num):
    string = f'{num:1b}'
    result = np.zeros(N - len(string), int)

    for ele in string:
        result = np.append(result, int(ele))

    return result

def get_fidelity_torch(vector1, vector2):
    overlap = torch.tensordot(vector1, vector2, dims=1)
    overlap_conj = torch.conj(overlap)
    fidelity = torch.real(overlap * overlap_conj)
    return fidelity

def get_energy_torch(matrix, vector):
    conj_vector = torch.conj(vector)
    energy = torch.real(torch.tensordot(conj_vector, torch.tensordot(matrix, vector, dims=1), dims=1)).to(torch.float64)
    return energy
    
# decimal number to binary array function
def D2B_with_digit(num,digit):
    string = f'{num:1b}'
    result = np.zeros(digit - len(string), int)

    for ele in string:
        result = np.append(result, int(ele))

    return result

# binary array to decimal function
def B2D(array):
    res = 0
    for ele in array:
        res = (res << 1) | ele
    return res

def spin_reflection(array):
    new_array = np.zeros(len(array),dtype=int)
    for i in range(len(array)):
        if array[i] == 1:
            new_array[i] = 0
        else:
            new_array[i] = 1
    return new_array

def spatial_reflection(array):
    new_array = array[::-1]
    return new_array

# start my main function

# initialize J matrix
J_matrix = np.zeros((N, N), dtype='float64')

for i in range(N):
    for j in range(N):
        if j >= (i + 1):
            J_matrix[i][j] = J_value/pow(abs(i-j),J_matrix_alpha)

In [3]:
# start to get what product states are having the same symmtries
loop_state_list = np.linspace(0,Q-1,num=Q,dtype=int)

couple_state_matrix = []
sign_matrix = []
# this is the final result for the states to consctruct basis 

for n in range(Q):  # n is an unused index
    
    # couple_num_list would be my result for each state we loop
    
    if len(loop_state_list)!=0: # loop for couple number for one state
                
        couple_num_list = [] # this is the list to store all the number that couple to the number we are looping
        couple_num_list = np.array(couple_num_list, dtype=int)
        state_num = loop_state_list[0]
        couple_num_list = np.append(couple_num_list,state_num)
        
        for m in range(Q): # loop for spin and spatial for all states inside the couple number list
            
            if m==0:
                
                couple_num_list_old =couple_num_list
                
                for i in range(len(couple_num_list)):
                    number = couple_num_list[i]
                    array = D2B(number)

                    # spin reflection 
                    spin_relfection_array = spin_reflection(array)
                    spin_number = B2D(spin_relfection_array)
                    couple_num_list =np.append(couple_num_list,spin_number)



                    # # spatial reflection 
                    spatial_relfection_array = spatial_reflection(array)
                    spatial_number = B2D(spatial_relfection_array)
                    couple_num_list =np.append(couple_num_list,spatial_number)
                    
                    # remove the repeated element 
                    couple_num_list = np.unique(couple_num_list)
            else:
                
                if len(couple_num_list_old) != len(couple_num_list):
                    # update couple_num_list_old first 
                    couple_num_list_old =couple_num_list
                    
                    for i in range(len(couple_num_list)):
                        number = couple_num_list[i]
                        array = D2B(number)

                        # spin reflection 
                        spin_relfection_array = spin_reflection(array)
                        spin_number = B2D(spin_relfection_array)
                        couple_num_list =np.append(couple_num_list,spin_number)
                        
                        
                        # # spatial reflection 
                        spatial_relfection_array = spatial_reflection(array)
                        spatial_number = B2D(spatial_relfection_array)
                        couple_num_list = np.append(couple_num_list,spatial_number)
                        
                        # remove the repeated element 
                        couple_num_list = np.unique(couple_num_list)
                        
        # now we have final couple_num_list
        couple_state_matrix.append(couple_num_list.tolist())
        
        # remove the ones already found 
        for j in range(len(couple_num_list)):
            loop_state_list = np.delete(loop_state_list, np.where(loop_state_list ==couple_num_list[j] ))

In [4]:
# initialize H matrix
couple_dimen = len(couple_state_matrix)

HA_matrix = np.zeros((couple_dimen, couple_dimen), dtype='float64')
HB_matrix = np.zeros((couple_dimen, couple_dimen), dtype='float64')

# loop each basis to fill in the diagonal elements for HA matrix 

for i in range(couple_dimen):
    
    state_array = np.array( couple_state_matrix[i] )
    
    # loop each element inside the array 
    normalization = len(state_array)
    
    # energy total for all the product states inside one basis
    
    e_tot = 0
    for j in range(len(state_array)):
        
        state = D2B(state_array[j])
        
        # loop over N sites to change the state vector to either 1 or -1
        
        for k in range(N):
            # 1 is spin down , 0 is spin up
            if state[k] == 1:
                state[k] = -1
            else:
                state[k] = 1

        # now get the energy, loop all the sites and then pairs
        
        for m in range(N):
            for q in range(N):
                e_temp = J_matrix[m][q] * state[m] * state[q]
                e_tot = e_tot + e_temp

    HA_matrix[i][i] = e_tot/normalization

# generate vector form for each basis
vector_form_basis_matrix = np.zeros((couple_dimen,Q))

for i in range(couple_dimen):
    
    state_array = np.array( couple_state_matrix[i] )
    normalization = np.sqrt(len(state_array))
    
    for j in state_array:
        vector_form_basis_matrix[i][j] = 1
        
    # put back normalization factor
    vector_form_basis_matrix[i] = vector_form_basis_matrix[i]/normalization
    
# loop each basis to fill in HB matrix 

for i in range(couple_dimen):

    state_array = np.array( couple_state_matrix[i] )

    # this is the actual normalization unlike before since it will couple to some other states
    # so it can not be like just the square like we did before
    normalization = np.sqrt(len(state_array))

    # loop each product state to generate N other states, i.e. generaete the state after operator acting on it
    result_num_list = []
    result_num_list = np.array(result_num_list,dtype=int)
    for k in range(len(state_array)):
        
        spin = D2B(state_array[k])

        # start fliping the spins
        for j in range(N):
            if spin[j] == 1:
                # if it's spin down, raise to spin up
                spin[j] = 0
                new_num = B2D(spin)
                result_num_list = np.append(result_num_list,new_num)
                # change it back for next operation
                spin[j] = 1
            else:
                # if it's spin down, raise to spin up
                spin[j] = 1
                new_num = B2D(spin)
                result_num_list = np.append(result_num_list,new_num)
                # change it back for next operation
                spin[j] = 0
        
     
    # generate the vector form for the state after operrator acting on it
    
    vector_final_state = np.zeros(Q)
    
    for j in result_num_list:
        vector_final_state[j] = vector_final_state[j] +1
        
    # put back normalization factor 
    vector_final_state = vector_final_state/normalization
    
    # calculate dot product to put into inside the matrix
    for j in range(couple_dimen):
        
        dot_product = np.dot(vector_final_state,vector_form_basis_matrix[j])
        HB_matrix[i][j] = dot_product

In [5]:
# Compute the smallest eigenvalue and eigenvector
smallest_eigenvalue, smallest_eigenvector = eigsh(HA_matrix + B * HB_matrix, k=1, which='SA')

true_gs_energy = smallest_eigenvalue[0]
true_gs_vec = smallest_eigenvector[:, 0]

# Compute the largest eigenvalue
largest_eigenvalue = eigs(HA_matrix + B * HB_matrix, k=1, which='LR', return_eigenvectors=False)

E_max = largest_eigenvalue[0].real

In [6]:
print('true ground state energy is', true_gs_energy)

true ground state energy is -7.887943169537985


In [7]:
# get  ground state with numerically infinite large B field
# Compute the smallest eigenvalue and eigenvector
smallest_eigenvalue, smallest_eigenvector = eigsh( HB_matrix, k=1, which='SA')
temp_vec = smallest_eigenvector[:, 0]

In [8]:
# move to gpu
true_gs_vec = torch.tensor(true_gs_vec.astype(np.complex128)).to(device=device)
temp_vec = torch.tensor(temp_vec.astype(np.complex128)).to(device=device)

In [9]:
# get fidelity for t= 0s
fidelity_t_0 = get_fidelity_torch(true_gs_vec,temp_vec)
print('fidelity before optim is', fidelity_t_0)

fidelity before optim is tensor(0.0014, device='cuda:0', dtype=torch.float64)


In [10]:
HA_matrix = torch.tensor(HA_matrix, dtype=torch.complex128).to(device=device)
HB_matrix = torch.tensor(HB_matrix, dtype=torch.complex128).to(device=device)
target_H = HA_matrix + B * HB_matrix

In [11]:
def lambda_evolve(x_A, x_B):
    global fidelity
    p = temp_vec.clone()
    # Loop through each time step
    for i in range(len(x_A)):
        H_matrix = x_A[i] * HA_matrix + x_B[i] * HB_matrix
        exp_imgH_matrix = torch.matrix_exp(1j * H_matrix)
        new_p = torch.mv(exp_imgH_matrix, p)
        p = new_p

        # Get energy
        energy = get_energy_torch(target_H, p)

    # Compute fidelity
    fidelity = get_fidelity_torch(true_gs_vec, p)
    print('fidelity is',fidelity)    
    return energy.item()

In [12]:
def find_m(a, b, max_ratio):
    
    a = np.abs(a)
    b =np.abs(b)
    
    # Start with an initial value for m
    m = 1
    
    # Define a function to calculate the ratio (a/m + b/m) / (a+b)
    def ratio_function_1(m):
        return ( (a*a*b)/(24*m*m) ) / (a + b)
    
    def ratio_function_2(m):
        return ( (a*b*b)/(12*m*m) ) / (a + b)

    for m in range(1,200):
        
        if ratio_function_2(m)<max_ratio and ratio_function_1(m)<max_ratio:
            #print(ratio_function_1(m),ratio_function_2(m))
            best_m = m
            break
    return best_m


In [13]:
# Length of the array
length = len(x)
# Split the array into two halves
x_A = x[:length//2]
x_B =x[:length//2] * x[length//2:]

The following cell computes the energy and fidelity using the input x in MTE form, serving as a check to ensure the input is correct.

In [14]:
print('energy is',lambda_evolve(x_A,x_B))

fidelity is tensor(0.9277, device='cuda:0', dtype=torch.float64)
energy is -7.743098451542654


Here, we input a m value , details are in https://doi.org/10.48550/arXiv.2408.03251 supplemental file
- N=12 
- 0.007 for 31 layers
- 0.004 for 41 layers
- 0.0024 for 51 layers
- 0.0016 for 61 layers
- 0.0011 for 71 layers
- 0.00085 for 81


In [15]:
# get p_list from ratio
p_list =[]
for i in range(len(x_A)):
    p = find_m(x_A[i], x_B[i], 0.007)     
    p_list.append(p)  

In [16]:
def get_expand_amps(gamma,beta,layers):
    
    if layers ==1:
        gamma_series = np.array([gamma/2,gamma/2])
        beta_series = np.array([beta])
        
    else:
        gamma_series = np.ones(layers-1)*(gamma/layers)
        # Add an element to the first index
        gamma_series = np.insert(gamma_series,0,(gamma/(2*layers)))
        gamma_series = np.insert(gamma_series,len(gamma_series),(gamma/(2*layers)))
        
        beta_series = np.ones(layers)*(beta/layers) # plus one for the first one 
    
    return gamma_series,beta_series
    

from itertools import chain
# get qaoa like input
new_x_A_list =[]
new_x_B_list =[]
for i in range(len(x_A)):
    gamma = x_A[i]
    beta = x_B[i]
    index_of_power = p_list[i]
    #print(index_of_power)
    new_x_A,new_x_B = get_expand_amps(gamma,beta,index_of_power)
    
    # append result
    new_x_A_list.append(new_x_A)
    new_x_B_list.append(new_x_B)

#new_x_A_list = list(chain.from_iterable(new_x_A_list))
new_x_B_list = list(chain.from_iterable(new_x_B_list))


flattened_new_x_A_list = []

for i, sublist in enumerate(new_x_A_list):
    if i > 0:
        flattened_new_x_A_list[-1] = flattened_new_x_A_list[-1] + sublist[0]
        flattened_new_x_A_list.extend(sublist[1:])
    else:
        flattened_new_x_A_list.extend(sublist)


In [17]:
from itertools import chain
# get qaoa like input
new_x_A_list =[]
new_x_B_list =[]
for i in range(len(x_A)):
    gamma = x_A[i]
    beta = x_B[i]
    index_of_power = p_list[i]
    #print(i)
    #print(index_of_power)
    new_x_A,new_x_B = get_expand_amps(gamma,beta,index_of_power)
    
    # append result
    new_x_A_list.append(new_x_A)
    new_x_B_list.append(new_x_B)

#new_x_A_list = list(chain.from_iterable(new_x_A_list))
new_x_B_list = list(chain.from_iterable(new_x_B_list))


flattened_new_x_A_list = []

for i, sublist in enumerate(new_x_A_list):
    if i > 0:
        flattened_new_x_A_list[-1] = flattened_new_x_A_list[-1] + sublist[0]
        flattened_new_x_A_list.extend(sublist[1:])
    else:
        flattened_new_x_A_list.extend(sublist)

x_1 = np.concatenate((flattened_new_x_A_list,new_x_B_list))
x_1 = torch.tensor(x_1, dtype=torch.float64, device=device, requires_grad=False)

In [18]:
print('number of QAOA layers is',len(flattened_new_x_A_list))

number of QAOA layers is 31


In [19]:
def lambda_evolve_qaoa_for_optim(x):

    location = len(x) // 2 + 1

    x_A = x[:location]
    x_B = x[location:]    
    #global fidelity

    p = temp_vec.clone()

    number_of_layers = len(x_B)
    for i in range(number_of_layers):
        exp_imgHA_matrix = torch.matrix_exp(1j * HA_matrix * x_A[i])
        exp_imgHB_matrix = torch.matrix_exp(1j * HB_matrix * x_B[i])

        H_eff = torch.matmul(exp_imgHB_matrix, exp_imgHA_matrix)
        new_p = torch.mv(H_eff, p)
        p = new_p
        
    # Apply the extra x_A
    exp_imgHA_matrix = torch.matrix_exp(1j * HA_matrix * x_A[-1])
    new_p = torch.mv(exp_imgHA_matrix, p)
    p = new_p
    energy = get_energy_torch(target_H, p)
    
    # fidelity = get_fidelity_torch(true_gs_vec, p)
    # fidelity = fidelity.item()
    # print('fidelity is {:.10f}'.format(fidelity))
    # print(torch.norm(p).item())
    
    return energy

In [20]:
def get_fidelity_list(x):
    global fidelity_list
    fidelity_list = []
    if len(x)%2==1:
        location = len(x) // 2 + 1
    else:
        location = len(x)//2
    x_A = x[:location]
    x_B = x[location:]    
    global fidelity

    p = temp_vec.clone().to(torch.complex128)  # Create a copy of temp_vec

    number_of_layers = len(x_B)
    if len(x_A) == len(x_B):
        # print('y')
        for i in range(number_of_layers):
            exp_imgHA_matrix = torch.matrix_exp(1j * HA_matrix * x_A[i])
            exp_imgHB_matrix = torch.matrix_exp(1j * HB_matrix * x_B[i])

            H_eff = torch.matmul(exp_imgHB_matrix, exp_imgHA_matrix)
            new_p = torch.matmul(H_eff, p)
            p = new_p
            fidelity = get_fidelity_torch(true_gs_vec, p)
            fidelity = fidelity.item()
            fidelity_list.append(fidelity) 
    else:
        for i in range(number_of_layers):
            #print('yy')
            exp_imgHA_matrix = torch.matrix_exp(1j * HA_matrix * x_A[i])
            exp_imgHB_matrix = torch.matrix_exp(1j * HB_matrix * x_B[i])

            H_eff = torch.matmul(exp_imgHB_matrix, exp_imgHA_matrix)
            new_p = torch.matmul(H_eff, p)
            p = new_p
            fidelity = get_fidelity_torch(true_gs_vec, p)
            fidelity = fidelity.item()
            fidelity_list.append(fidelity) 
            
        # Apply the extra x_A
        exp_imgHA_matrix = torch.matrix_exp(1j * HA_matrix * x_A[-1])
        new_p = torch.matmul(exp_imgHA_matrix, p)
        p = new_p
        fidelity = get_fidelity_torch(true_gs_vec, p)
        fidelity = fidelity.item()
        fidelity_list.append(fidelity)  

In [21]:
def train_lambda_evolve(lr, gradient_threshold, filename, checkpoint_filename):
    # Initialize x if checkpoint does not exist
    if os.path.isfile(checkpoint_filename):
        checkpoint = torch.load(checkpoint_filename)
        x = checkpoint['x']
        optimizer = optim.Adam([x], lr=lr)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
    else:
        x = np.concatenate((flattened_new_x_A_list,new_x_B_list))
        x = torch.tensor(x, dtype=torch.float64, device=device, requires_grad=True)
        # start the code
        optimizer = optim.Adam([x], lr=lr)
        epoch = 0

    with open(filename, 'a') as f:  # Use 'a' to append to the log file if it exists
        while True:
            optimizer.zero_grad()
            energy = lambda_evolve_qaoa_for_optim(x)
            energy.backward()
            optimizer.step()

            gradient_norm = x.grad.norm().item()
            if (epoch + 1) % 1 == 0:
                result_str = f"Epoch {epoch + 1}, Energy: {energy:.6f}, Energy ratio: {energy/true_gs_energy:.6f}, Gradient Norm: {gradient_norm:.6f}\n"
                f.write(result_str)
                print(result_str, end='')
                
                # Append latest x values to the end of the file
                x_values = ','.join([str(val.item()) for val in x.detach().cpu().numpy()])
                f.write(f"Latest x values at epoch {epoch + 1}: {x_values}\n")
                #print(x_values)
                # Save checkpoint
                torch.save({
                    'epoch': epoch,
                    'x': x,
                    'optimizer_state_dict': optimizer.state_dict(),
                }, checkpoint_filename)

            if gradient_norm < gradient_threshold:
                break

            epoch += 1
            f.flush()

In [2]:
filename = f'QAOA_training_log_N={N}_B={B}_layers_{np.sum(p_list)}.txt'
checkpoint_filename = f'QAOA_checkpoint_log_N={N}_B={B}_layers_{np.sum(p_list)}.pt'
# Execute the following line to run the optimization.
###########################################

train_lambda_evolve(0.01, 0.0001, filename, checkpoint_filename) # learning rate is 0.01 and gradient_threshold is 0.0001

In [ ]:
# Example usage based on previously optimized data.
best_30= np.array([0.09883560217862536,0.29694661452567783,-0.04223160455803097,0.34181833185331073,0.445708802140215,0.06395875308781014,-0.11792400368569012,0.10450382791716804,0.45155063856821553,0.36773186562921584,0.3863144467491393,0.39546759474137805,0.404117194948392,0.3967876132794703,0.40748532879952903,0.4099682623145645,0.40656032453149993,0.4125831926125769,0.3926871711459888,0.4130993265682991,0.4203272478053987,0.422596280764216,0.4229697451117508,0.43198254299121674,0.4497398463801568,0.45602406596982586,0.4876138744248867,0.534394230545182,0.5543904072606549,0.6271244896223498,1.5167029563561208e-05,0.4906107243574948,1.1078658145481528,0.9035715524291998,0.24417628356738635,0.4413182695174657,0.7183517990437245,0.43254690242630117,0.4299498753313533,0.216668514283277,0.21673022670131928,0.21574089603285987,0.20957406675069212,0.20081762127126224,0.19750743053319525,0.19584105814594038,0.190340461508735,0.18713658074980472,0.18085387666004657,0.16948479532136507,0.17731147459550622,0.17214489959363782,0.17318536422139827,0.16991322232129275,0.17008332423645686,0.1652976299524704,0.15444635288438988,0.15705058885756187,0.1537392636682203,0.13881566146195073,0.08520288908426696])
best_30 = torch.tensor(best_30, dtype=torch.float64, device=device, requires_grad=False)
best_40= np.array([0.08881331901466091,0.20254001129362792,0.24688487477037965,0.28302925664456213,0.2848820234049403,0.2660245719451472,0.374766081173048,0.06431200038651136,-0.0608531065744281,0.08130627174407143,0.4156532134983728,0.34745130689167936,0.34631921805344956,0.3375800372112298,0.3242013207429833,0.3372144424188058,0.3588928802164133,0.3705324112499944,0.3810622128238857,0.3885167426488866,0.39844906724448137,0.4019157548540339,0.41218599498050507,0.41614163518812536,0.4165136150239514,0.4089295159127287,0.401108472087148,0.39835036734262336,0.4074636908403127,0.42188869857300826,0.4129301982664925,0.41177241637976386,0.40729633217424543,0.43054826354083,0.4508635131685727,0.46410595849413133,0.5063852033835946,0.5494401769194854,0.567086434793776,0.6247390181548891,6.746421110907407e-05,0.5140990470938749,0.3503380067226817,1.8572707132079727,0.25001432998937884,0.21136932729853308,0.18492777709502156,0.35586506020101705,0.6635980532307426,0.5757455634514476,0.3455680802619926,0.20094258852901678,0.19624133326039417,0.18638050345769977,0.17414677750097998,0.1669682636408392,0.17628003493106556,0.18054087003357353,0.18390548713408136,0.18447899397739304,0.18591629387563607,0.18550485808028688,0.1830903180342377,0.1834495917302014,0.18320293634987184,0.18253822676843245,0.17838874003137642,0.17459778516576274,0.16814250620110413,0.17022111148954094,0.16564478275879155,0.1638668057250036,0.1607868427183128,0.16235156765931685,0.1645419676426412,0.15781543456698666,0.15243991951605224,0.1562192267972927,0.15256539042834527,0.13236262197763693,0.07430557266090056])
best_40 = torch.tensor(best_40, dtype=torch.float64, device=device, requires_grad=False)
best_50= np.array([0.08202858153292118,0.27033773130163047,0.04726475971313934,-0.10368191723038658,0.056835232625533506,0.34516291772960866,0.2641919653947468,0.2824210789231795,0.38811092399271063,0.07858827171035965,-0.061015802739330965,0.0793511055124828,0.4076720877732578,0.32732681602194985,0.32721665776209363,0.32290728426629034,0.3073305102928346,0.30473011349578427,0.312375739167832,0.31401451215619613,0.33022635080157503,0.347303661742102,0.3514754725888425,0.3560128895907037,0.3632019631063949,0.3698581091869817,0.37183544827030757,0.37317146582776906,0.3773937241609838,0.37830693567774437,0.37448166019869406,0.376926014016424,0.37240270124700636,0.362087483263841,0.36517567305338183,0.3785328261009834,0.37756628702474676,0.3757191721459734,0.38491419965082246,0.3927663435591142,0.39460924268261416,0.38896012391910983,0.4054603986158366,0.42407131260906056,0.436475078950618,0.49900568625481273,0.571196121891741,0.6104590213056507,0.6042606645196058,0.6071903106756192,0.00012414087511123534,0.5346562075268407,0.5049671489347731,0.6632433400547064,0.5272045023043983,0.5012373567358367,0.25104700584115924,0.2268166241632921,0.22039413393726037,0.3170830684440872,0.7348626390058447,0.5442898244640234,0.3660123640065208,0.19884133562152703,0.19123932085919504,0.1870098761535026,0.1741588917793626,0.16201305733673352,0.16094172304521376,0.16075212974863734,0.16061918314496992,0.16829226667179614,0.17000692323561195,0.16916506569238415,0.17024358506552928,0.1705698595797983,0.1695282682019988,0.1677201624095296,0.16840418398206647,0.1685549202662053,0.1662892736469787,0.16330504732605775,0.16066980231647146,0.1551411017064267,0.15076082846017622,0.15449528522054126,0.15702750379981856,0.15170761510966413,0.1497256630797447,0.14816515959326435,0.14389230072870318,0.14080923901622086,0.135972523004814,0.1412283949687429,0.13290922811976083,0.13157248893698967,0.1364824384971456,0.1303150189703873,0.11755555152255949,0.09773266659415607,0.05135425464600847])
best_50 = torch.tensor(best_50, dtype=torch.float64, device=device, requires_grad=False)
best_60= np.array([ 0.06324719379437627,0.1473716594160256,0.25102311196385574,-0.03275432400399373,-0.02859591983106859,0.31300741232086826,0.24477415476125466,0.22997098076543457,0.2277591667251185,0.3396279292723425,0.06353274826292464,-0.013309454384126791,-0.04631006697879746,0.10474508092410711,0.33622537339215675,0.2723483914817253,0.27451333911286024,0.28746114694459485,0.29522306852454633,0.28958834849512155,0.27548785116354385,0.2700226978553042,0.299248647345696,0.32173063644254224,0.3312646203616534,0.33706537836173156,0.3410480872401923,0.320518293979827,0.2983267118589641,0.29623015581253614,0.3005504287830846,0.3242337630964402,0.3534339685065611,0.36003051731913355,0.358573636148054,0.3690533954940621,0.3773851977218561,0.37628881975349765,0.3788366553287864,0.3809124443172497,0.36664913893740175,0.36899989712282655,0.3927333447853013,0.4022328551129756,0.39008668907342464,0.38680842497387774,0.39325087583714824,0.39028233735061485,0.3914544524015571,0.40186616425934546,0.4151329042561052,0.4344628143428997,0.48761727705451324,0.5225635483260939,0.5327506540171775,0.5313307717532096,0.5481747856582048,0.5743001892148237,0.5694139410773734,0.5709183684838275,0.0033531825490181522,0.5328225872065516,0.3726208383575648,0.5330702567353865,0.9729347768144168,0.4487922983562509,0.2586179057252713,0.2139561266700648,0.17932570587158997,0.1734832118616349,0.36182712444700976,0.3159727642425698,0.5345613851220813,0.4159635756743863,0.2991275851552877,0.17102992999372835,0.16388107373066807,0.16512437156151416,0.16769566056573848,0.1639162819360573,0.1522120586386143,0.14128682884551216,0.14535571612642575,0.15831851511123654,0.1631581816643168,0.16543105699506355,0.16632730410738508,0.16164724628251803,0.1452741856467702,0.1371256049195222,0.13717814221417687,0.13998794058601924,0.15458776413996753,0.1622427139123582,0.16094161437218887,0.16120041405578878,0.16484143584347485,0.1636647967872986,0.16226425793204335,0.16304273628152335,0.15851941639846115,0.14998913448620455,0.15451251504402427,0.15862480771468132,0.15497302992012754,0.14820062283113367,0.1466077491793567,0.14304871466919428,0.13373663018801232,0.12802565921149206,0.12475311976138155,0.12199427316176999,0.12798300110236469,0.13429903991493575,0.13048073033163843,0.12264157427453835,0.11565392950110827,0.1138690155738625,0.10753542788266177,0.09173414300514689,0.048231576391915075])
best_60 = torch.tensor(best_60, dtype=torch.float64, device=device, requires_grad=False)
best_70= np.array([0.05013361299216526,0.11713905899313692,0.15493695714930078,0.18704079692964928,0.2086986281535457,0.2241520882218094,0.2315867340862514,0.23704426589590494,0.2510306132558461,0.26261350190598987,0.26291254551124166,0.2449045368997676,0.20181387411691376,0.04109938502361471,-0.010921756851566683,0.011115324503567777,0.11361259461322747,0.2568053072635486,0.2678802651968086,0.28043365403580084,0.282759715116457,0.2792827789158674,0.2676223075474714,0.27121663114423655,0.28108655551348016,0.2779633623407621,0.2685850836378745,0.2717644631529878,0.28962570276266286,0.29950201645268604,0.29649354176830045,0.292907213864896,0.29852088603587945,0.307418523547124,0.30900500827322464,0.3067995468967261,0.2967923993406924,0.2825291647027424,0.29560557952741273,0.3172541670239796,0.32289414783763437,0.31080985934017313,0.29119864864527806,0.2949904090260997,0.31158732827354363,0.3323287362724592,0.34991163805477093,0.3554512327000839,0.36154337770355083,0.36997664478871306,0.3745331686654994,0.3785821830147145,0.38622966565721034,0.38709489340127995,0.38813114352832484,0.39475020141305145,0.3991030401909274,0.40777683161047346,0.4158517686371176,0.41387300126254994,0.40582932859685866,0.40842690273263493,0.40685822221594975,0.40961348959646643,0.4150430169566799,0.42408285067126633,0.49226858623908964,0.5532016252011881,0.5666924877409893,0.5147264737079338,0.09214233652676866,0.5460472810716175,0.3809261017837478,0.3147289552738696,0.2691564805038066,0.2413992551531472,0.21844214141814963,0.1983225971396291,0.18957420571415856,0.18733240425156497,0.18072272294209954,0.16740899573574583,0.14000915730631777,0.2719474328072933,0.5105206183252895,0.5420467261006625,0.3221482933887958,0.15555561424812453,0.15130966983475924,0.15700674117692595,0.1569443725149228,0.15342481108915237,0.14549576750354834,0.1389512155127716,0.14253039193097283,0.14231253005986724,0.13620064751913707,0.13155731984753277,0.13660383330376138,0.14259767571174048,0.14223229777279142,0.1383113311151506,0.13737510150566187,0.14061349793996164,0.1418370372761194,0.1399839986871965,0.13655339568118408,0.12829526381159378,0.12591074837348476,0.13523800117065657,0.14068912317292237,0.1385568312241086,0.12906140992984697,0.12352580612272705,0.12774802616709433,0.1336012248298797,0.14053321051240666,0.14324148572489506,0.14288524125547256,0.14433419913647663,0.14426875712597623,0.14180120178171263,0.13986209898214066,0.13658656788517612,0.13054794053523006,0.12672308202677976,0.12308307894542429,0.11903773946259061,0.1173562961086389,0.11373851252161826,0.10840941437948154,0.10405513843053778,0.10100529845518377,0.09564052369014064,0.0920048087920932,0.08558116430354869,0.08544183545850055,0.08973231004609589,0.08508157111921852,0.0717415625442238,0.04131235746511464])
best_70 = torch.tensor(best_70, dtype=torch.float64, device=device, requires_grad=False)
best_80= np.array([0.030574082179259877,0.05084334633462071,0.09669739277208426,0.1345342515743292,0.15630571815915417,0.14921695689732206,0.12627133482694622,0.16650592937283168,0.17779588708990926,0.12529076786496607,0.1626375935924034,0.2070229212281155,0.1564364606393484,0.1607388033763727,0.07737519082209679,-0.003228520025742783,-0.007129563460275362,0.07627722047518848,0.1567939606562988,0.20303692941977936,0.2529986140301044,0.25238856229593587,0.17671639502200176,0.17059531322206672,0.2489947436130967,0.2682867124468809,0.2778118460749751,0.28305417673453065,0.2846943589079045,0.28349193843310483,0.2807287474549748,0.2783860220669825,0.2795291429044335,0.2798544342776568,0.27748664196720674,0.2794062375909371,0.2854797701474966,0.28710578197934467,0.28413769683772805,0.28179156614413314,0.28707735452502864,0.2988145855460415,0.30717780879951784,0.3077577187481952,0.3069555639903366,0.30607067755891953,0.30186799306782536,0.30032988033368874,0.30230719515668053,0.2971937366080384,0.29154868968833164,0.30430922637297514,0.3191180322555967,0.3195764216992493,0.3183304258019918,0.3169878838469653,0.31294141965813954,0.3165448393867777,0.3195386826069848,0.3274444047160869,0.34522841145892985,0.36270402164986765,0.37799047535542357,0.386445118910997,0.3884215339201508,0.39050534564990397,0.39591473734043564,0.4006579277143043,0.40604501149746897,0.4082101386654194,0.4039046842123338,0.40298277419593814,0.4049031106392406,0.40928556851350945,0.42071816372648757,0.43021525279403244,0.46708637850964335,0.5117661509526434,0.5234794874676824,0.4444656412648462,0.06799800348820806,0.4334536796946855,0.32937284275483786,0.33484391521837287,0.29402662972250215,0.24235465306686133,0.17255891653535346,0.16529864981768452,0.18635285728506984,0.1457851728881418,0.10999535055673985,0.16092354378930734,0.15542568355041952,0.08585547361272676,0.2263975058691485,0.3446115849620437,0.6542489110273112,0.36665545400985367,0.2185287533643568,0.10209474268547145,0.15527833389054088,0.16103871958596122,0.13972034224052726,0.08092955509336158,0.13086060938716204,0.1487043748646528,0.15254744117384467,0.15348464329316808,0.15216149155328498,0.14935948362188875,0.1457777991056799,0.14185766870893105,0.1396392359016095,0.13860933083200488,0.1360435553438115,0.1336367352124105,0.13483671818364865,0.13567482132669526,0.133686821789748,0.13117633707461923,0.13018870489913117,0.13348053016489578,0.13759356465928355,0.1382466730372829,0.13670428531623247,0.13555077039386132,0.1333259755647205,0.13060659618025564,0.130059224556431,0.12878625003453592,0.12382113480018392,0.12368565590897794,0.1298287149843957,0.13132471258455702,0.12894894480162025,0.12716179615858833,0.12374508222094817,0.12097862814848444,0.1202634951928484,0.1186241701668569,0.12072710670225802,0.12362654433657444,0.12529243515958857,0.1246270733912611,0.12112677647583363,0.11679837775451114,0.11354716676163806,0.1109955604745495,0.10862021372192879,0.10617829270941763,0.10188074271814684,0.09591574768147303,0.09104027872921851,0.08530698198302093,0.08110165293474161,0.0760664838339432,0.07214740506045358,0.07211557019939345,0.06931856444022175,0.059231561840442364,0.03105179779867607])
best_80 = torch.tensor(best_80, dtype=torch.float64, device=device, requires_grad=False)

In [ ]:
# result from previously optimized data.
lambda_evolve_qaoa_for_optim(best_80)